In [31]:

from qiskit_mps_initializer.datatypes import QuantumState
from qiskit import QuantumCircuit
import numpy as np
import matplotlib.pyplot as plt

In [37]:
qc= QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.draw(output='mpl')

MissingOptionalLibraryError: "The 'matplotlib' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install matplotlib'."

'2.1.0'

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'datatypes',
 'helpers',
 'utils']

In [ ]:
mps.draw(figsize=(12, 12))

NameError: name 'mps' is not defined